In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

#helpers
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')

In [ ]:
train['Title'] = train.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
test['Title'] = test.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
train['Title'] = train['Title'].fillna('Mr').map( {'Sir':'c','Countess':'a','Ms':'a','Mme':'a','Lady':'a','Mlle':'a','Mrs':'a','Miss':'a','Master':'b','Col':'c','Major':'c','Dr':'c','Mr':'d','Jonkheer':'d','Don':'d','Rev':'d','Capt':'d','Dona':'a'} ).astype('category')
test['Title'] = test['Title'].fillna('Mr').map( {'Sir':'c','Countess':'a','Ms':'a','Mme':'a','Lady':'a','Mlle':'a','Mrs':'a','Miss':'a','Master':'b','Col':'c','Major':'c','Dr':'c','Mr':'d','Jonkheer':'d','Don':'d','Rev':'d','Capt':'d','Dona':'a'} ).astype('category')

In [ ]:
#train['Embarked'] = train['Embarked'].fillna('S').map( {'C': 0, 'Q': 1, 'S': 2} ).astype(int)
#train['Sex'] = train['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
#test['Embarked'] = test['Embarked'].fillna('S').map({'C': 0, 'Q': 1, 'S': 2}).astype(int)
#test['Sex'] = test['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
#test.fillna(12, inplace=True)

In [ ]:
#train['Pclass'] = train.Pclass.astype('category')
train['Sex'] = train.Sex.astype('category')
train['SibSp'] = train.SibSp.astype('category')
train['Parch'] = train.Parch.astype('category')
train['Embarked'] = train.Embarked.astype('category')
train['Age_group'] = pd.cut(train['Age'].fillna(0), 10)

In [ ]:
#test['Pclass'] = train.Pclass.astype('category')
test['Sex'] = train.Sex.astype('category')
test['SibSp'] = train.SibSp.astype('category')
test['Parch'] = train.Parch.astype('category')
test['Embarked'] = train.Embarked.astype('category')
test['Age'] = test.Age.fillna(np.mean(test.Age))
test['Age_group'] = pd.cut(test['Age'].fillna(0), 10)

In [ ]:
X_train = train[['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked', 'Age_group', 'Title']]
Y_train = train["Survived"]
X_test  = test[['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked', 'Age_group', 'Title']]
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [ ]:
X_test.info()

In [ ]:
# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
print(logreg.score(X_train, Y_train))

In [ ]:
plot_learning_curve(logreg, "Logistic Regression", X_train, Y_train, ylim=(0.7, 1.01), cv=cv, n_jobs=4)
plt.show()

In [ ]:
pd.Series(logreg.coef_[0])

In [ ]:
np.std(X_train, axis=0).reset_index(drop=True)*pd.Series(logreg.coef_[0])

In [ ]:
submit = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": Y_pred
    })
submit.to_csv('logreg_best.csv', index = False)
#submit

In [ ]:
pca = PCA(n_components=12)
pca.fit(X_train)
for i in range(len(pca.explained_variance_ratio_)):
    print(np.sum(pca.explained_variance_ratio_[0:i]))  
#print(pca.singular_values_)

In [ ]:
logistic = linear_model.LogisticRegression()
pca = decomposition.PCA(n_components=15)
pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])
plot_learning_curve(pipe, "PCA+LogReg", X_train, Y_train, ylim=(0.7, 1.01), cv=cv, n_jobs=4)
plt.show()

In [ ]:
pipe.fit(X_train, Y_train)
Y_pred = pipe.predict(X_test)
print(pipe.score(X_train, Y_train))